In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact

# Load data
filename = '../npz/BTCUSD_1T.npz'
data = np.load(filename, allow_pickle=True)['data']
loaded_df = pd.DataFrame(data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])

# Convert 'timestamp' to datetime and set it as the index
loaded_df['timestamp'] = pd.to_datetime(loaded_df['timestamp'])
loaded_df.set_index('timestamp', inplace=True)

# Convert all price and volume columns to numeric format
for column in ['open', 'high', 'low', 'close', 'volume']:
    loaded_df[column] = pd.to_numeric(loaded_df[column], errors='coerce')

from scipy.signal import butter, filtfilt
def lowpass_filter(data, cutoff=0.1, fs=1.0, order=2):
    nyquist = 0.5 * fs
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    y = filtfilt(b, a, data)
    return y

# Create interactive widgets
n_slider = widgets.IntSlider(min=1000, max=30 * 1000, step=1000, value=5000, description='n')
w1_slider = widgets.IntSlider(min=1, max=1500, step=1, value=30, description='w1')
w2_slider = widgets.IntSlider(min=1, max=3000, step=1, value=60, description='w2')
cutoff_slider = widgets.FloatSlider(min=0.001, max=0.3, step=0.0005, value=0.01, description='cutoff', readout_format='.3f')
# Function to update the plot
def update_plot2(n, w1, w2, cutoff):
    # Select the last n rows
    df = loaded_df.tail(n)
    w1_rolling = df['close'].rolling(window=2*w1).mean()
    w1_shifted = w1_rolling.shift(-w1)
    w2_rolling = df['close'].rolling(window=2*w2).mean()
    w2_shifted = w2_rolling.shift(-w2)
    diff = (w2_shifted - w1_shifted).dropna()
    smoothed = lowpass_filter(diff.values, cutoff=cutoff)
    # plt.figure(figsize=(15, 5))
    # df['close'].plot(label='Close price')
    # w1_rolling.plot(label=f'w1_rolling')
    # w2_rolling.plot(label=f'w2_rolling')
    # plt.legend()
    # plt.figure(figsize=(15, 5))
    # w1_shifted.plot(label=f'w1_shifted')
    # w2_shifted.plot(label=f'w2_shifted')
    # plt.legend()
    plt.figure(figsize=(15, 5))
    smoothed_series = pd.Series(smoothed, index=diff.index)

    # ax = diff.plot(label='diff')
    # # Explicitly plot smoothed_series on the same axes using its index and values
    # ax.plot(smoothed_series.index, smoothed_series.values, label='smoothed')
    #print(smoothed_series)
    
    plt.plot(diff, label=f'diff')
    plt.plot(smoothed_series, label='smoothed')
    
    plt.legend()
    # plt.figure(figsize=(15, 5))
    # # plot the same but without dates on X axis
    # plt.plot(diff.values, label='diff')
    # plt.plot(smoothed, label='smoothed')
    # plt.legend()
    # plt.figure(figsize=(15, 5))
    # plt.plot(smoothed, label='smoothed')
    # plt.legend()
    # plt.figure(figsize=(15, 5))
    # plt.plot(diff.values - smoothed, label='diff - smoothed')
    # plt.legend()
    
    plt.show()

# Use interact to create the interactive plot
interact(update_plot2, n=n_slider, w1=w1_slider, w2=w2_slider, cutoff=cutoff_slider)